In [ ]:
from addict import Dict
from pathlib import Path
import numpy as np

data_dir = Path("/datadrive/glaciers/mappingvis/")
process_dir = data_dir / "processed"
args = Dict({
    "device": "cuda",
    "out_dir": data_dir / "preds"
})

if args.out_dir.exists():
    shutil.rmtree(args.out_dir)
    
args.out_dir.mkdir()

In [ ]:
import torch
from unet import Unet

state = torch.load(data_dir / "model.pt")
model = Unet(13, 3, 4).to(args.device)
model.load_state_dict(state)
model.eval()

In [ ]:
from data import GlacierDataset
from torch.utils.data import DataLoader

paths = {}
for v in ["x", "y"]:
    paths[v] = list(process_dir.glob(v + "*"))
    paths[v].sort()

ds = GlacierDataset(paths["x"], paths["y"])

In [ ]:
import re

for i in range(len(ds)):
    x, y = ds[i]
    ix = re.search("[0-9]+", str(ds.x_paths[i])).group(0)
    print(f"saving {i + 1}/{len(ds)}...", end="\r", flush=True)

    with torch.no_grad():
        y_hat = model(x.unsqueeze(0).to(args.device))
        y_hat = y_hat.cpu()
        np.save(args.out_dir / f"y_hat-{i}.npy", y_hat[0])
        np.save(args.out_dir / f"y-{i}.npy", x)
        np.save(args.out_dir / f"x-{i}.npy", y)    